# Evaluate model for Vietnamese Stock Env

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())


import pandas as pd
import pandas_ta as ta
import gym
import mt4_hst
import stock_env
from stock_env.utils import *
from stock_env.envs.vn_stock_env import VietnamStockEnv, VietnamStockV2Env
from stable_baselines3.common.env_checker import check_env

/Users/manbnm/miniconda3/envs/tradingenv/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
ticker = 'FPT'
path = "stock_env/datasets/"
df = mt4_hst.read_hst(path + ticker + "1440.hst")

In [6]:
import numpy as np
from gym import spaces

class MultiStockEnv(gym.Env):
    def __init__(
        self, 
        tickers, 
        feature_extractor,
        data_folder_path,
        tick_size: float = 0.05,
        lot_size: int = 100,
        max_trade_lot: int = 5,
        max_lot: int = 10,
        kappa: float = 1e-4,
        init_cash: float = 2e4,
        random_seed: int = None,
    ):
        self.tickers = tickers
        self.feature_extractor = feature_extractor
        self.data_folder_path = data_folder_path
        self.n_ticker = len(self.tickers)
        self.tick_size = tick_size
        self.lot_size = lot_size
        self.max_trade_lot = max_trade_lot
        self.max_lot = max_lot
        self.kappa = kappa
        self.init_total_cash = init_cash
        self.init_ticker_cash = self.init_total_cash / self.n_ticker
        self.random_seed = random_seed
        
        self._create_envs()
        self.obs_dim = sum([env.observation_space.shape[0] for env in self.envs])
        self.observation_space = spaces.Box(
            low=-np.inf, 
            high=np.inf, 
            shape=(self.obs_dim + 1,),
            dtype=np.float64
        )
        self.action_space = spaces.MultiDiscrete([env.action_space.n for env in self.envs])
        
        self._start_tick = 0
        self._end_tick = self.df.shape[0] - 1
        self._current_tick = None
        self.done = None
        self.total_reward = None
        self.positions = None
        self.histories = None
    
    def _create_envs(self):
        self.envs = []
        df = pd.DataFrame()
        for ticker in self.tickers:
            _df = mt4_hst.read_hst(self.data_folder_path + ticker + "1440.hst")
            _df.sort_values(by='time', inplace=True)
            _df['ticker'] = ticker
            df = pd.concat([df, _df])

        self.df = df.pivot(columns=['ticker']).swaplevel(0, 1, axis=1)
        self.df.dropna(inplace=True)
        
        for ticker in self.tickers:
            _env = VietnamStockV2Env(
                df = self.df[ticker],
                feature_extractor = self.feature_extractor,
                tick_size = self.tick_size,
                lot_size = self.lot_size,
                max_trade_lot = self.max_trade_lot,
                max_lot = self.max_lot,
                kappa = self.kappa,
                init_cash = self.init_ticker_cash,
                random_seed = self.random_seed,
            )
            self.envs.append(_env)
    
    @property
    def nav(self):
        self._current_tick = np.random.randint(self._start_tick, self._end_tick)
        return sum([env.nav for env in self.envs])
    
    @property
    def cash(self):
        return sum([env.cash for env in self.envs])
    
    def reset(self, current_tick='random'):
        #TODO
        if current_tick == 'random':
            self._current_tick = np.random.randint(self._start_tick, self._end_tick)
        elif isinstance(current_tick, int):
            self._current_tick = current_tick
        else:
            raise NotImplementedError
        obs = np.concatenate([env.reset(self._current_tick) for env in self.envs])
        obs = np.append(obs, self.cash).astype(np.float64)
        return obs
    
    def step(self):
        pass